In [1]:
import pandas as pd
import math
from scipy.stats import levene, shapiro, kruskal
from statsmodels.formula.api import ols
from scikit_posthocs import posthoc_dunn
import statsmodels.formula.api as smf

In [2]:
# define file paths
file = 'path'

In [10]:
# import files
total_data = pd.read_excel(file+'/total_data_revised.xlsx')
county_data = pd.read_excel(file+'/county_data_revised.xlsx')

In [4]:
# data check
county_data.head()

,region_code,state,region,type,total_deaths,expected_deaths,excess_deaths,excess_deaths per 100k,peak,POP_CAGR,GRDP_CAGR,Income,The elderly,Education,Unemployment,Race_white,Race_black
0,US55123,Wisconsin,"Vernon County, WI","Pop-Growth, Econ-Decline",30.611111,23.832633,6.778478,21.932669,7,0.333860,-0.358675,60.041,19.525743,23.512555,2.7,96.674144,0.243831
1,US48209,Texas,"Hays County, TX",Growing,130.194444,107.668185,22.526259,8.672325,3,4.378620,5.164925,79.336,10.988587,38.673646,3.0,82.658744,4.131007
2,US1005,Alabama,"Barbour County, AL",Shrinking,30.000000,29.687519,0.312481,1.236664,8,-0.974886,-1.312064,38.649,19.420441,11.153098,4.1,46.299848,47.666427
3,US48203,Texas,"Harrison County, TX","Pop-Growth, Econ-Decline",69.472222,55.065716,14.406507,20.758928,11,0.273632,-2.215375,56.645,16.900459,21.124477,4.5,71.577982,20.708672
4,US48091,Texas,"Comal County, TX",Growing,140.805556,125.691813,15.113743,8.750407,8,4.154444,4.948369,93.487,18.082131,39.601859,3.5,87.100543,2.172964


## 1. Descriptive statistics

#### (1) All Counties (for Table 1)

In [11]:
total_data.sum()

region_code               US1129US1129US1129US1129US1129US1129US1129US11...
state                     AlabamaAlabamaAlabamaAlabamaAlabamaAlabamaAlab...
region                    Washington County, ALWashington County, ALWash...
urban_type                ShrinkingShrinkingShrinkingShrinkingShrinkingS...
time                      2020/032020/042020/052020/062020/072020/082020...
total_deaths                                                        6984343
expected_deaths                                              6054965.494768
excess_deaths                                                 929377.505232
excess_deaths per 100k                                        575729.275872
CumulativeExcessDeaths                                      11609648.553874
dtype: object

In [12]:
# Mean, Standard Deviation, Minimum, and Maximum, standard error and 95% confidence interval 
temp_1 = total_data.describe().T
temp_1 = temp_1.drop(['25%', '50%', '75%'], axis = 1)
temp_1['count'] = temp_1['count'].astype(int)
temp_1['ste'] = temp_1['std'].astype(float)/math.sqrt(1142)
temp_1['low_ci'] = temp_1['mean'] - 1.96*temp_1['ste']
temp_1['hi_ci'] = temp_1['mean'] + 1.96*temp_1['ste']
temp_1.round(2)

,count,mean,std,min,max,ste,low_ci,hi_ci
total_deaths,41112,169.89,352.27,10.00,14778.00,10.42,149.45,190.32
expected_deaths,41112,147.28,289.31,9.58,6547.87,8.56,130.50,164.06
excess_deaths,41112,22.61,97.21,-124.25,8319.90,2.88,16.97,28.24
excess_deaths per 100k,41112,14.00,24.39,-101.91,281.70,0.72,12.59,15.42
CumulativeExcessDeaths,41112,282.39,257.07,-580.82,2177.61,7.61,267.48,297.30


In [13]:
county_data.sum()

region_code               US55123US48209US1005US48203US48091US19193US101...
state                     WisconsinTexasAlabamaTexasTexasIowaAlabamaNew ...
region                    Vernon County, WIHays County, TXBarbour County...
type                      Pop-Growth, Econ-DeclineGrowingShrinkingPop-Gr...
total_deaths                                                  194009.527778
expected_deaths                                               168193.485966
excess_deaths                                                  25816.041812
excess_deaths per 100k                                         15992.479885
peak                                                                   7776
POP_CAGR                                                         473.611884
GRDP_CAGR                                                       1238.702869
Income                                                            71735.346
The elderly                                                    20572.322477
Education   

In [7]:
# Mean, Standard Deviation, Minimum, and Maximum, standard error and 95% confidence interval 
temp_1 = county_data.describe().T
temp_1 = temp_1.drop(['25%', '50%', '75%'], axis = 1)
temp_1['count'] = temp_1['count'].astype(int)
temp_1['ste'] = temp_1['std'].astype(float)/math.sqrt(1142)
temp_1['low_ci'] = temp_1['mean'] - 1.96*temp_1['ste']
temp_1['hi_ci'] = temp_1['mean'] + 1.96*temp_1['ste']
temp_1.round(2)

,count,mean,std,min,max,ste,low_ci,hi_ci
total_deaths,1142,169.89,341.58,15.56,6667.33,10.11,150.07,189.70
expected_deaths,1142,147.28,288.64,12.70,5481.40,8.54,130.54,164.02
excess_deaths,1142,22.61,54.92,-3.09,1185.94,1.63,19.42,25.79
excess_deaths per 100k,1142,14.00,8.53,-16.13,60.49,0.25,13.51,14.50
peak,1142,6.81,2.64,0.00,13.00,0.08,6.66,6.96
POP_CAGR,1142,0.41,0.96,-2.47,4.38,0.03,0.36,0.47
GRDP_CAGR,1142,1.08,2.27,-9.79,18.43,0.07,0.95,1.22
Income,1142,62.82,16.72,28.24,153.72,0.49,61.85,63.79
The elderly,1142,18.01,4.48,7.72,57.59,0.13,17.75,18.27
Education,1142,26.33,10.90,6.38,76.30,0.32,25.70,26.96


#### (2) Each urban types

A. Growing cities

In [16]:
# total_deaths, expected_deaths,excess_deaths, excess_deaths per 100k, number of peak, standard error and 95% confidence interval 
temp_growing = county_data[county_data['type']=='Growing'].describe().T
temp_growing = temp_growing.drop(['25%', '50%', '75%'], axis = 1)
temp_growing['count'] = temp_growing['count'].astype(int)
temp_growing['ste'] = temp_growing['std'].astype(float)/math.sqrt(692)
temp_growing['low_ci'] = temp_growing['mean'] - 1.96*temp_growing['ste']
temp_growing['hi_ci'] = temp_growing['mean'] + 1.96*temp_growing['ste']
temp_growing.round(2)

,count,mean,std,min,max,ste,low_ci,hi_ci
total_deaths,691,233.01,416.26,15.56,6667.33,15.82,201.99,264.02
expected_deaths,691,202.06,350.55,12.70,5481.40,13.33,175.94,228.18
excess_deaths,691,30.95,68.12,-1.26,1185.94,2.59,25.87,36.02
excess_deaths_per_100k,691,11.62,6.55,-5.69,42.17,0.25,11.13,12.10
peak,691,5.87,2.65,0.00,13.00,0.10,5.67,6.07
POP_CAGR,691,0.97,0.74,0.03,4.38,0.03,0.92,1.03
GRDP_CAGR,691,2.04,1.55,0.00,17.66,0.06,1.92,2.15
Income,691,69.69,16.48,36.71,153.72,0.63,68.46,70.92
The_elderly,691,17.23,5.03,7.72,57.59,0.19,16.85,17.60
Education,691,30.60,11.07,10.11,76.30,0.42,29.77,31.42


B. Sririnking cities

In [17]:
# total_deaths, expected_deaths,excess_deaths, excess_deaths per 100k, number of peak, standard error and 95% confidence interval 
temp_shrinking = county_data[county_data['type']=='Shrinking'].describe().T
temp_shrinking = temp_shrinking.drop(['25%', '50%', '75%'], axis = 1)
temp_shrinking['count'] = temp_shrinking['count'].astype(int)
temp_shrinking['ste'] = temp_shrinking['std'].astype(float)/math.sqrt(241)
temp_shrinking['low_ci'] = temp_shrinking['mean'] - 1.96*temp_shrinking['ste']
temp_shrinking['hi_ci'] = temp_shrinking['mean'] + 1.96*temp_shrinking['ste']
temp_shrinking.round(2)

,count,mean,std,min,max,ste,low_ci,hi_ci
total_deaths,243,63.39,55.23,17.58,301.17,3.56,56.42,70.36
expected_deaths,243,54.43,48.11,13.46,254.72,3.10,48.36,60.50
excess_deaths,243,8.96,8.13,-0.80,53.10,0.52,7.94,9.99
excess_deaths_per_100k,243,19.11,9.87,-4.91,60.49,0.64,17.86,20.35
peak,243,8.39,1.76,3.00,13.00,0.11,8.17,8.61
POP_CAGR,243,-0.62,0.41,-2.47,-0.04,0.03,-0.67,-0.57
GRDP_CAGR,243,-1.71,1.47,-9.79,-0.07,0.09,-1.90,-1.53
Income,243,49.54,9.99,28.24,107.01,0.64,48.28,50.80
The_elderly,243,19.27,3.07,8.65,36.13,0.20,18.88,19.65
Education,243,18.99,6.28,6.38,41.49,0.40,18.19,19.78


C. Pop-Growth, Econ-Decline

In [18]:
# total_deaths, expected_deaths,excess_deaths, excess_deaths per 100k, number of peak, standard error and 95% confidence interval 
temp_PGED = county_data[county_data['type']=='Pop-Growth, Econ-Decline'].describe().T
temp_PGED = temp_PGED.drop(['25%', '50%', '75%'], axis = 1)
temp_PGED['count'] = temp_PGED['count'].astype(int)
temp_PGED['ste'] = temp_PGED['std'].astype(float)/math.sqrt(36)
temp_PGED['low_ci'] = temp_PGED['mean'] - 1.96*temp_PGED['ste']
temp_PGED['hi_ci'] = temp_PGED['mean'] + 1.96*temp_PGED['ste']
temp_PGED.round(2)

,count,mean,std,min,max,ste,low_ci,hi_ci
total_deaths,39,94.58,67.20,24.72,279.31,11.20,72.63,116.53
expected_deaths,39,81.64,59.15,17.17,235.46,9.86,62.31,100.96
excess_deaths,39,12.95,9.05,1.31,43.85,1.51,9.99,15.90
excess_deaths_per_100k,39,15.23,8.81,4.80,43.49,1.47,12.35,18.10
peak,39,6.87,1.85,4.00,11.00,0.31,6.27,7.48
POP_CAGR,39,0.67,0.55,0.05,2.37,0.09,0.50,0.85
GRDP_CAGR,39,-1.42,1.06,-4.72,-0.24,0.18,-1.77,-1.07
Income,39,63.35,14.26,37.53,106.71,2.38,58.69,68.01
The_elderly,39,17.14,4.26,9.16,28.77,0.71,15.75,18.53
Education,39,26.11,7.51,16.07,44.98,1.25,23.66,28.57


D. Pop-Decline, Econ-Growth

In [19]:
# total_deaths, expected_deaths,excess_deaths, excess_deaths per 100k, number of peak, standard error and 95% confidence interval 
temp_PDEG = county_data[county_data['type']=='Pop-Decline, Econ-Growth'].describe().T
temp_PDEG = temp_PDEG.drop(['25%', '50%', '75%'], axis = 1)
temp_PDEG['count'] = temp_PDEG['count'].astype(int)
temp_PDEG['ste'] = temp_PDEG['std'].astype(float)/math.sqrt(173)
temp_PDEG['low_ci'] = temp_PDEG['mean'] - 1.96*temp_PDEG['ste']
temp_PDEG['hi_ci'] = temp_PDEG['mean'] + 1.96*temp_PDEG['ste']
temp_PDEG.round(2)

,count,mean,std,min,max,ste,low_ci,hi_ci
total_deaths,169,82.30,183.08,16.69,1802.42,13.92,55.02,109.58
expected_deaths,169,71.96,159.57,14.04,1533.16,12.13,48.18,95.74
excess_deaths,169,10.34,24.19,-3.09,269.25,1.84,6.74,13.95
excess_deaths_per_100k,169,16.14,9.68,-16.13,43.39,0.74,14.70,17.59
peak,169,8.36,1.89,2.00,13.00,0.14,8.08,8.64
POP_CAGR,169,-0.44,0.27,-1.39,-0.02,0.02,-0.48,-0.40
GRDP_CAGR,169,1.79,2.15,0.13,18.43,0.16,1.47,2.11
Income,169,53.66,7.71,37.82,71.56,0.59,52.51,54.81
The_elderly,169,19.63,2.69,12.47,29.59,0.20,19.23,20.03
Education,169,19.48,5.90,10.40,38.85,0.45,18.60,20.36


## 2. Kruskal-Wallis test

#### (1) Monthly excss death per 100,000 people

A. Test for levene, shapiro

In [22]:
# Rename columns to handle spaces if necessary
county_data.columns = county_data.columns.str.replace(' ', '_')

# Checking for homogeneity of variances using Levene's test
levene_test = levene(county_data['excess_deaths_per_100k'][county_data['type'] == 'Growing'],
                     county_data['excess_deaths_per_100k'][county_data['type'] == 'Shrinking'],
                     county_data['excess_deaths_per_100k'][county_data['type'] == 'Pop-Growth, Econ-Decline'],
                     county_data['excess_deaths_per_100k'][county_data['type'] == 'Pop-Decline, Econ-Growth'])

# Check if data is normally distributed using Shapiro-Wilk test
anova_model = ols('excess_deaths_per_100k ~ type', data=county_data).fit()
shapiro_test = shapiro(anova_model.resid)

levene_test_pvalue = levene_test.pvalue
shapiro_test_pvalue = shapiro_test.pvalue

print(f"Levene's test p-value: {levene_test_pvalue:.4f}")
print(f"Shapiro-Wilk test p-value: {shapiro_test_pvalue:.4f}")

Levene's test p-value: 0.0000
Shapiro-Wilk test p-value: 0.0000


B. test for kruska-wallis and posthoc

In [23]:
# Perform Kruskal-Wallis test
kruskal_test = levene(county_data['excess_deaths_per_100k'][county_data['type'] == 'Growing'],
                     county_data['excess_deaths_per_100k'][county_data['type'] == 'Shrinking'],
                     county_data['excess_deaths_per_100k'][county_data['type'] == 'Pop-Growth, Econ-Decline'],
                     county_data['excess_deaths_per_100k'][county_data['type'] == 'Pop-Decline, Econ-Growth'])

print(f"Kruskal-Wallis test statistic: {kruskal_test.statistic:.4f}, p-value: {kruskal_test.pvalue:.4f}")

# Perform Dunn's post-hoc test with Bonferroni correction
posthoc_results = posthoc_dunn(county_data, val_col='excess_deaths_per_100k', group_col='type', p_adjust='bonferroni')

# Print post-hoc results
print(posthoc_results)

# Convert posthoc_results to a styled HTML table for better visualization (if needed)
styled_posthoc_results = posthoc_results.style.background_gradient(cmap='viridis').set_caption("Dunn's Post-hoc Test Results")

# Display the styled table in a Jupyter Notebook
styled_posthoc_results

Kruskal-Wallis test statistic: 23.2783, p-value: 0.0000
                               Growing  Pop-Decline, Econ-Growth  \
Growing                   1.000000e+00              1.239788e-09   
Pop-Decline, Econ-Growth  1.239788e-09              1.000000e+00   
Pop-Growth, Econ-Decline  1.058170e-01              1.000000e+00   
Shrinking                 4.767144e-29              1.614971e-02   

                          Pop-Growth, Econ-Decline     Shrinking  
Growing                                   0.105817  4.767144e-29  
Pop-Decline, Econ-Growth                  1.000000  1.614971e-02  
Pop-Growth, Econ-Decline                  1.000000  4.969007e-02  
Shrinking                                 0.049690  1.000000e+00  


,Growing,"Pop-Decline, Econ-Growth","Pop-Growth, Econ-Decline",Shrinking
Growing,1.000000,0.000000,0.105817,0.000000
"Pop-Decline, Econ-Growth",0.000000,1.000000,1.000000,0.016150
"Pop-Growth, Econ-Decline",0.105817,1.000000,1.000000,0.049690
Shrinking,0.000000,0.016150,0.049690,1.000000


#### (2) Number of peaks

A. Test for levene, shapiro

In [24]:
# Rename columns to handle spaces if necessary
county_data.columns = county_data.columns.str.replace(' ', '_')

# Checking for homogeneity of variances using Levene's test
levene_test = levene(county_data['peak'][county_data['type'] == 'Growing'],
                     county_data['peak'][county_data['type'] == 'Shrinking'],
                     county_data['peak'][county_data['type'] == 'Pop-Growth, Econ-Decline'],
                     county_data['peak'][county_data['type'] == 'Pop-Decline, Econ-Growth'])

# Check if data is normally distributed using Shapiro-Wilk test
anova_model = ols('peak ~ type', data=county_data).fit()
shapiro_test = shapiro(anova_model.resid)

levene_test_pvalue = levene_test.pvalue
shapiro_test_pvalue = shapiro_test.pvalue

print(f"Levene's test p-value: {levene_test_pvalue:.4f}")
print(f"Shapiro-Wilk test p-value: {shapiro_test_pvalue:.4f}")

Levene's test p-value: 0.0000
Shapiro-Wilk test p-value: 0.0000


B. test for kruska-wallis and posthoc

In [25]:
# Perform Kruskal-Wallis test
kruskal_test = levene(county_data['peak'][county_data['type'] == 'Growing'],
                     county_data['peak'][county_data['type'] == 'Shrinking'],
                     county_data['peak'][county_data['type'] == 'Pop-Growth, Econ-Decline'],
                     county_data['peak'][county_data['type'] == 'Pop-Decline, Econ-Growth'])

print(f"Kruskal-Wallis test statistic: {kruskal_test.statistic:.4f}, p-value: {kruskal_test.pvalue:.4f}")

# Perform Dunn's post-hoc test with Bonferroni correction
posthoc_results = posthoc_dunn(county_data, val_col='peak', group_col='type', p_adjust='bonferroni')

# Print post-hoc results
print(posthoc_results.to_string())

# Convert posthoc_results to a styled HTML table for better visualization (if needed)
styled_posthoc_results = posthoc_results.style.background_gradient(cmap='viridis').set_caption("Dunn's Post-hoc Test Results")

# Display the styled table in a Jupyter Notebook
styled_posthoc_results

Kruskal-Wallis test statistic: 30.4701, p-value: 0.0000
                               Growing  Pop-Decline, Econ-Growth  Pop-Growth, Econ-Decline     Shrinking
Growing                   1.000000e+00              7.272355e-28                  0.344657  2.425303e-37
Pop-Decline, Econ-Growth  7.272355e-28              1.000000e+00                  0.001885  1.000000e+00
Pop-Growth, Econ-Decline  3.446573e-01              1.884961e-03                  1.000000  9.864416e-04
Shrinking                 2.425303e-37              1.000000e+00                  0.000986  1.000000e+00


,Growing,"Pop-Decline, Econ-Growth","Pop-Growth, Econ-Decline",Shrinking
Growing,1.000000,0.000000,0.344657,0.000000
"Pop-Decline, Econ-Growth",0.000000,1.000000,0.001885,1.000000
"Pop-Growth, Econ-Decline",0.344657,0.001885,1.000000,0.000986
Shrinking,0.000000,1.000000,0.000986,1.000000


## 3. A mixed effect model

#### (1) Monthly excss death per 100,000 people

A. null model

In [28]:
# Fit a null mixed-effects model
# Null model includes only the intercept and the random effect (grouping variable)

null_model = smf.mixedlm(
    'excess_deaths_per_100k ~ 1',  # Only intercept
    county_data, 
    groups=county_data['state'],
    re_formula='~1'  # Random intercept
)
null_result = null_model.fit(reml=False)

# Display the null model results
print(null_result.summary())

# Extract 2LL, AIC, and BIC
two_ll = -2 * null_result.llf
aic = null_result.aic
bic = null_result.bic

# Print the results
print(f"2LL: {two_ll}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: excess_deaths_per_100k
No. Observations: 1142    Method:             ML                    
No. Groups:       44      Scale:              59.7142               
Min. group size:  5       Log-Likelihood:     -3996.5293            
Max. group size:  80      Converged:          Yes                   
Mean group size:  26.0                                              
-----------------------------------------------------------------------
              Coef.     Std.Err.      z       P>|z|    [0.025    0.975]
-----------------------------------------------------------------------
Intercept     13.388       0.653    20.518    0.000    12.109    14.667
Group Var     15.194       0.531                                       

2LL: 7993.058578882613
AIC: 7999.058578882613
BIC: 8014.180188053261


B. full model

In [29]:
# Fit a full mixed-effects model

model = smf.mixedlm(
    'excess_deaths_per_100k ~ POP_CAGR + GRDP_CAGR + Income + The_elderly + Education + Unemployment + Race_white + Race_black',  
    county_data, 
    groups=county_data['state'],
    re_formula='~1'
)
result = model.fit(reml=False)

# Display the results
print(result.summary())

# Extract 2LL, AIC, and BIC
two_ll = -2 * result.llf
aic = result.aic
bic = result.bic

# Print the results
print(f"2LL: {two_ll}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: excess_deaths_per_100k
No. Observations: 1142    Method:             ML                    
No. Groups:       44      Scale:              43.9292               
Min. group size:  5       Log-Likelihood:     -3813.3279            
Max. group size:  80      Converged:          Yes                   
Mean group size:  26.0                                              
----------------------------------------------------------------------
                  Coef.    Std.Err.     z      P>|z|   [0.025   0.975]
----------------------------------------------------------------------
Intercept         22.095      4.181    5.285   0.000   13.901   30.290
POP_CAGR          -0.907      0.376   -2.415   0.016   -1.643   -0.171
GRDP_CAGR         -0.237      0.108   -2.189   0.029   -0.449   -0.025
Income            -0.049      0.024   -2.088   0.037   -0.095   -0.003
The_elderly        0.351      0.056 

#### (2) Number of peaks

A. null model

In [30]:
# Fit a null mixed-effects model
# Null model includes only the intercept and the random effect (grouping variable)

null_model = smf.mixedlm(
    'peak ~ 1',  # Only intercept
    county_data, 
    groups=county_data['state'],
    re_formula='~1'  # Random intercept
)
null_result = null_model.fit(reml=False)

# Display the null model results
print(null_result.summary())

# Extract 2LL, AIC, and BIC
two_ll = -2 * null_result.llf
aic = null_result.aic
bic = null_result.bic

# Print the results
print(f"2LL: {two_ll}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: peak      
No. Observations: 1142    Method:             ML        
No. Groups:       44      Scale:              5.9828    
Min. group size:  5       Log-Likelihood:     -2676.8785
Max. group size:  80      Converged:          Yes       
Mean group size:  26.0                                  
---------------------------------------------------------
           Coef.  Std.Err.    z     P>|z|  [0.025  0.975]
---------------------------------------------------------
Intercept  6.678     0.180  37.147  0.000   6.326   7.030
Group Var  1.079     0.125                               

2LL: 5353.757046176354
AIC: 5359.757046176354
BIC: 5374.878655347003


B. full model

In [155]:
# Fit a full mixed-effects model

model = smf.mixedlm(
    'peak ~ POP_CAGR + GRDP_CAGR + Income + The_elderly + Education + Unemployment + Race_white + Race_black',  
    county_data, 
    groups=county_data['state'],
    re_formula='~1'
)
result = model.fit(reml=False)

# Display the results
print(result.summary())

# Extract 2LL, AIC, and BIC
two_ll = -2 * result.llf
aic = result.aic
bic = result.bic

# Print the results
print(f"2LL: {two_ll}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: peak      
No. Observations: 1142    Method:             ML        
No. Groups:       44      Scale:              2.8345    
Min. group size:  5       Log-Likelihood:     -2236.0766
Max. group size:  80      Converged:          Yes       
Mean group size:  26.0                                  
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept     3.586    1.006   3.565 0.000  1.614  5.558
POP_CAGR     -0.610    0.087  -6.999 0.000 -0.781 -0.439
GRDP_CAGR     0.051    0.027   1.863 0.063 -0.003  0.104
Income       -0.019    0.006  -3.317 0.001 -0.030 -0.008
The_elderly   0.137    0.014   9.700 0.000  0.109  0.164
Education    -0.082    0.008 -10.723 0.000 -0.097 -0.067
Unemployment -0.173    0.066  -2.610 0.009 -0.303 -0.043
Race_white    0.056    0.009   6.328 0.00